In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

from constants.companies import get_company_by_code

company_code = 'GL'
company = get_company_by_code(company_code=company_code)
company_id = company.company_id

In [ ]:
from catalog_connector import connection
df_truth = connection.sql(
    f"""
        select * from
        mlgold.dishes_forecasting_order_history
        where company_id = '{company_id}'
    """
).toPandas()

df_truth.head()

In [4]:
from dishes_forecasting.paths import PROJECT_DIR
forecasting_dir = PROJECT_DIR.parent.parent.parent / "Forecasting"
dishes_file = forecasting_dir / f"dishes_ml_forecast_{company_code.lower()}.csv"

In [5]:
import pandas as pd

df = pd.read_csv(dishes_file)

df_old_model= df.rename(
    columns={
        "variation_id": "product_variation_id",
        "year": "menu_year",
        "week": "menu_week",
        "variation_ratio": "variation_ratio_predicted",
    }
)

df_old_model = df_old_model.merge(
    df_truth,
    on=["product_variation_id", "menu_year", "menu_week", "company_id"],
    how="left",
).dropna(subset="variation_ratio")

df_old_model["variation_quantity_predicted"] = df_old_model["variation_ratio_predicted"] * df_old_model["total_weekly_qty"]
df_old_model["error"] = df_old_model["variation_quantity_predicted"] - df_old_model["product_variation_quantity"]
df_old_model["abs_error"] = df_old_model["error"].abs()
df_old_model["pct_error"] = df_old_model["error"] / df_old_model["product_variation_quantity"]
df_old_model["abs_pct_error"] = df_old_model["pct_error"].abs()

In [ ]:
df_old_model.groupby("run_id")["abs_error"].describe()["mean"].describe()

In [ ]:
df_new_model = connection.sql(
    f"""
        select * from
        mloutputs.ml_dishes_ratios
        where company_id = '{company_id}'
    """
).toPandas()

In [8]:
df_new_model = df_new_model.merge(
    df_truth,
    on=["product_variation_id", "menu_year", "menu_week", "company_id"],
    how="left",
).dropna(subset="variation_ratio")

df_new_model["variation_quantity_predicted"] = df_new_model["variation_ratio_prediction"] * df_new_model["total_weekly_qty"]
df_new_model["error"] = df_new_model["variation_quantity_predicted"] - df_new_model["product_variation_quantity"]
df_new_model["abs_error"] = df_new_model["error"].abs()
df_new_model["pct_error"] = df_new_model["error"] / df_new_model["product_variation_quantity"]
df_new_model["abs_pct_error"] = df_new_model["pct_error"].abs()

In [ ]:
df_new_model["abs_error"].describe()

In [ ]:
df_old_model = df_old_model[df_old_model["menu_week"] >= 27]
df_old_model["abs_error"].describe()

In [ ]:
df_old_model.head()

In [ ]:
df_old_model.groupby("run_timestamp")["abs_error"].describe()["mean"]

In [ ]:
df_new_model.groupby("created_at")["abs_error"].describe()["mean"]

In [ ]:
df_old_model.groupby("run_timestamp")["abs_pct_error"].describe()["50%"]

In [ ]:
df_new_model.groupby("created_at")["abs_pct_error"].describe()["50%"]

In [ ]:
df_new_model